In [1]:
# Install OpenNMT-py 3.x
!pip3 install OpenNMT-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.8/262.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 57.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  U

# Prepare Your Datasets
Please make sure you have completed the [first exercise](https://colab.research.google.com/drive/1rsFPnAQu9-_A6e2Aw9JYK3C8mXx9djsF?usp=sharing).

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Open the folder where you saved your prepapred datasets from the first exercise
# You might need to mount your Google Drive first
%cd /content/drive/MyDrive/Colab_Notebooks/Fictional_Neural_Translation/nmt
!ls

/content/drive/MyDrive/Colab_Notebooks/Fictional_Neural_Translation/nmt
compute-bleu.py  en-fr	 MT-Preparation  run	       source.vocab  target.vocab
config.yaml	 en-tkn  README		 source.model  target.model  train.log


# Create the Training Configuration File

The following config file matches most of the recommended values for the Transformer model [Vaswani et al., 2017](https://arxiv.org/abs/1706.03762). As the current dataset is small, we reduced the following values:
* `train_steps` - for datasets with a few millions of sentences, consider using a value between 100000 and 200000, or more! Enabling the option `early_stopping` can help stop the training when there is no considerable improvement.
* `valid_steps` - 10000 can be good if the value `train_steps` is big enough.
* `warmup_steps` - obviously, its value must be less than `train_steps`. Try 4000 and 8000 values.

Refer to [OpenNMT-py training parameters](https://opennmt.net/OpenNMT-py/options/train.html) for more details. If you are interested in further explanation of the Transformer model, you can check this article, [Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/).

In [4]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano
# Note here we are using some smaller values because the dataset is small
# For larger datasets, consider increasing: train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint

config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: /content/drive/MyDrive/Colab_Notebooks/Fictional_Neural_Translation/nmt/en-tkn/UN.en-tkn.tkn-filtered.tkn.subword.train
        path_tgt: /content/drive/MyDrive/Colab_Notebooks/Fictional_Neural_Translation/nmt/en-tkn/UN.en-tkn.en-filtered.en.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: /content/drive/MyDrive/Colab_Notebooks/Fictional_Neural_Translation/nmt/en-tkn/UN.en-tkn.tkn-filtered.tkn.subword.dev
        path_tgt: /content/drive/MyDrive/Colab_Notebooks/Fictional_Neural_Translation/nmt/en-tkn/UN.en-tkn.en-filtered.en.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source.model
tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.fren

# Stop training if it does not imporve after n validations
early_stopping: 4

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 1000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 3000

# Default: 10000 - Run validation after n steps
valid_steps: 1000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 1000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [5]:
# [Optional] Check the content of the configuration file
!cat config.yaml

# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: /content/drive/MyDrive/Colab_Notebooks/Fictional_Neural_Translation/nmt/en-tkn/UN.en-tkn.tkn-filtered.tkn.subword.train
        path_tgt: /content/drive/MyDrive/Colab_Notebooks/Fictional_Neural_Translation/nmt/en-tkn/UN.en-tkn.en-filtered.en.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: /content/drive/MyDrive/Colab_Notebooks/Fictional_Neural_Translation/nmt/en-tkn/UN.en-tkn.tkn-filtered.tkn.subword.dev
        path_tgt: /content/drive/MyDrive/Colab_Notebooks/Fictional_Neural_Translation/nmt/en-tkn/UN.en-tkn.en-filtered.en.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoo

# Build Vocabulary

For large datasets, it is not feasable to use all words/tokens found in the corpus. Instead, a specific set of vocabulary is extracted from the training dataset, usually betweeen 32k and 100k words. This is the main purpose of the vocabulary building step.

In [6]:
# Find the number of CPUs/cores on the machine
!nproc --all

8


In [7]:
# Build Vocabulary

# -config: path to your config.yaml file
# -n_sample: use -1 to build vocabulary on all the segment in the training dataset
# -num_threads: change it to match the number of CPUs to run it faster

!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 8

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-04-29 16:51:52,679 INFO] Counter vocab from -1 samples.
[2024-04-29 16:51:52,679 INFO] n_sample=-1: Build vocab on full datasets.
[2024-04-29 16:51:54,588 INFO] * Transform statistics for corpus_1(12.50%):
			* FilterTooLongStats(filtered=1)

[2024-04-29 16:51:54,592 INFO] * Transform statistics for corpus_1(12.50%):
			* FilterTooLongStats(filtered=6)

[2024-04-29 16:51:54,594 INFO] * Transform statistics for corpus_1(12.50%):
			* FilterTooLongStats(filtered=5)

[2024-04-29 16:51:54,608 INFO] Counters src: 191
[2024-04-29 16:51:54,608 INFO] Counters tgt: 224
Traceback (most recent call last):
  File "/usr/local/bin/onmt_build_vocab", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/onmt/bin/build_vocab.py", line 283, in main
    build_vocab_main(opts)
  File "/usr/local/lib/python3.10/dist-packages/onmt/bin/build_vocab.py", line 267, in build_vocab_main
    save_counter

From the **Runtime menu** > **Change runtime type**, make sure that the "**Hardware accelerator**" is "**GPU**".


In [8]:
# Check if the GPU is active
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-ea34662f-330d-80eb-ba89-a4e41748ca93)


In [9]:
# Check if the GPU is visable to PyTorch

import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

True
Tesla T4
Free GPU memory: 14999.0625 out of: 15102.0625


# Training

Now, start training your NMT model! 🎉 🎉 🎉

In [10]:
!rm -rf /content/drive/MyDrive/Colab_Notebooks/Fictional_Neural_Translation/nmt/models/

In [ ]:
# Train the NMT model
!onmt_train -config config.yaml

Streaming output truncated to the last 5000 lines.
[2024-04-30 12:32:04,698 INFO] Weighted corpora loaded so far:
			* corpus_1: 7983521
[2024-04-30 12:32:04,703 INFO] Weighted corpora loaded so far:
			* corpus_1: 7983522
[2024-04-30 12:32:04,709 INFO] Weighted corpora loaded so far:
			* corpus_1: 7983523
[2024-04-30 12:32:04,715 INFO] Weighted corpora loaded so far:
			* corpus_1: 7983524
[2024-04-30 12:32:04,721 INFO] Weighted corpora loaded so far:
			* corpus_1: 7983525
[2024-04-30 12:32:04,727 INFO] Weighted corpora loaded so far:
			* corpus_1: 7983526
[2024-04-30 12:32:04,734 INFO] Weighted corpora loaded so far:
			* corpus_1: 7983527
[2024-04-30 12:32:04,741 INFO] Weighted corpora loaded so far:
			* corpus_1: 7983528
[2024-04-30 12:32:04,747 INFO] Weighted corpora loaded so far:
			* corpus_1: 7983529
[2024-04-30 12:32:04,753 INFO] Weighted corpora loaded so far:
			* corpus_1: 7983530
[2024-04-30 12:32:04,759 INFO] Weighted corpora loaded so far:
			* corpus_1: 7983531
[20

In [ ]:
# For error debugging try:
!dmesg -T

[Thu Apr 18 00:05:36 2024] Linux version 6.1.58+ (builder@d2af7d2f4328) (Chromium OS 16.0_pre484197_p20230405-r10 clang version 16.0.0 (/var/tmp/portage/sys-devel/llvm-16.0_pre484197_p20230405-r10/work/llvm-16.0_pre484197_p20230405/clang 2916b99182752b1aece8cc4479d8d6a20b5e02da), LLD 16.0.0) #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
[Thu Apr 18 00:05:36 2024] Command line: BOOT_IMAGE=/syslinux/vmlinuz.A init=/usr/lib/systemd/systemd boot=local rootwait ro noresume loglevel=7 console=tty1 console=ttyS0 security=apparmor virtio_net.napi_tx=1 nmi_watchdog=0 csm.disabled=1 loadpin.exclude=kernel-module,firmware modules-load=loadpin_trigger firmware_class.path=/var/lib/nvidia/firmware module.sig_enforce=0 dm_verity.error_behavior=3 dm_verity.max_bios=-1 dm_verity.dev_wait=1 i915.modeset=1 cros_efi loadpin.enabled=0 root=/dev/dm-0 "dm-mod.create=vroot,,,ro,0 4077568 verity 0 PARTUUID=F635C99B-4144-1A41-A53F-3DB030D436A1 PARTUUID=F635C99B-4144-1A41-A53F-3DB030D436A1 4096 4096 509696

# Translation

Translation Options:
* `-model` - specify the last model checkpoint name; try testing the quality of multiple checkpoints
* `-src` - the subworded test dataset, source file
* `-output` - give any file name to the new translation output file
* `-gpu` - GPU ID, usually 0 if you have one GPU. Otherwise, it will translate on CPU, which would be slower.
* `-min_length` - [optional] to avoid empty translations
* `-verbose` - [optional] if you want to print translations

Refer to [OpenNMT-py translation options](https://opennmt.net/OpenNMT-py/options/translate.html) for more details.

In [ ]:
# Translate the "subworded" source file of the test dataset
# Change the model name, if needed.
!onmt_translate -model models/model.fren_step_3000.pt -src UN.en-fr.fr-filtered.fr.subword.test -output UN.en.translated -gpu 0 -min_length 1

[2024-04-18 01:02:50,113 INFO] Loading checkpoint from models/model.fren_step_3000.pt
[2024-04-18 01:02:51,985 INFO] Loading data into the model
[2024-04-18 01:04:18,399 INFO] PRED SCORE: -0.2286, PRED PPL: 1.26 NB SENTENCES: 2000
Time w/o python interpreter load/terminate:  88.3059823513031


In [ ]:
# Check the first 5 lines of the translation file
!head -n 5 UN.en.translated

In [ ]:
# If needed install/update sentencepiece
!pip3 install --upgrade -q sentencepiece

# Desubword the translation file
!python3 MT-Preparation/subwording/3-desubword.py target.model UN.en.translated

In [ ]:
# Check the first 5 lines of the desubworded translation file
!head -n 5 UN.en.translated.desubword

RESOLUTION 61/164
4. Takes note with appreciation of the efforts of the regional commissions to implement the World Programme of Action and to follow up the World Conference in their respective regions, in coordination with the regional meetings of Ministers of youth and non-governmental organizations, as well as in providing advisory services to support the such activities in each region, and encourages them to continue their efforts in this regard;
RESOLUTION 57/220
Recognizing also that a universal, rule-based, open, non-discriminatory and equitable multilateral trading system, as well as meaningful trade liberalization, can substantially stimulate development worldwide, benefiting countries at all stages of development, reaffirming its commitment to trade liberalization and to ensure that trade are fully contribute to the promotion of economic growth and development, and in this regard welcoming the decisions taken by the World Trade Organization and the World Trade Organization as

In [ ]:
# Desubword the target file (reference) of the test dataset
# Note: You might as well have split files *before* subwording during dataset preperation,
# but sometimes datasets have tokeniztion issues, so this way you are sure the file is really untokenized.
!python3 MT-Preparation/subwording/3-desubword.py target.model UN.en-fr.en-filtered.en.subword.test

Done desubwording! Output: UN.en-fr.en-filtered.en.subword.test.desubword


In [ ]:
# Check the first 5 lines of the desubworded reference
!head -n 5 UN.en-fr.en-filtered.en.subword.test.desubword

RESOLUTION 61/164
4. Takes note with appreciation of the work by the regional commissions to implement the World Programme of Action and to follow up the World Conference in their respective regions, in coordination with regional meetings of ministers responsible for youth and regional non-governmental youth organizations, and to provide advisory services to support national youth policies and programmes in each region, and encourages them to continue to do so;
RESOLUTION 57/220
Recognizing also that a universal, rule-based, open, non-discriminatory and equitable multilateral trading system, as well as meaningful trade liberalization, can substantially stimulate development worldwide, benefiting countries at all stages of development, reaffirming its commitment to trade liberalization and to ensuring that trade plays its full part in promoting economic growth, employment and development for all, welcoming in that regard the decisions taken by the World Trade Organization to place the n

# MT Evaluation

There are several MT Evaluation metrics such as BLEU, TER, METEOR, COMET, BERTScore, among others.

Here we are using BLEU. Files must be detokenized/desubworded beforehand.

In [ ]:
# Download the BLEU script
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py

--2024-04-18 01:04:28--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py’

compute-bleu.py     100%[===================>]     957  --.-KB/s    in 0s      

2024-04-18 01:04:28 (23.8 MB/s) - ‘compute-bleu.py’ saved [957/957]



In [ ]:
# Install sacrebleu
!pip3 install sacrebleu

In [ ]:
# Evaluate the translation (without subwording)
!python3 compute-bleu.py UN.en-fr.en-filtered.en.subword.test.desubword UN.en.translated.desubword

# More Features and Directions to Explore

Experiment with the following ideas:
* Icrease `train_steps` and see to what extent new checkpoints provide better translation, in terms of both BLEU and your human evaluation.

* Check other MT Evaluation mentrics other than BLEU such as [TER](https://github.com/mjpost/sacrebleu#ter), [WER](https://blog.machinetranslation.io/compute-wer-score/), [METEOR](https://blog.machinetranslation.io/compute-bleu-score/#meteor), [COMET](https://github.com/Unbabel/COMET), and [BERTScore](https://github.com/Tiiiger/bert_score). What are the conceptual differences between them? Is there special cases for using a specific metric?

* Continue training from the last model checkpoint using the `-train_from` option, only if the training stopped and you want to continue it. In this case, `train_steps` in the config file should be larger than the steps of the last checkpoint you train from.
```
!onmt_train -config config.yaml -train_from models/model.fren_step_3000.pt
```

* **Ensemble Decoding:** During translation, instead of adding one model/checkpoint to the `-model` argument, add multiple checkpoints. For example, try the two last checkpoints. Does it improve quality of translation? Does it affect translation seepd?

* **Averaging Models:** Try to average multiple models into one model using the [average_models.py](https://github.com/OpenNMT/OpenNMT-py/blob/master/onmt/bin/average_models.py) script, and see how this affects translation quality.
```
python3 average_models.py -models model_step_xxx.pt model_step_yyy.pt -output model_avg.pt
```
* **Release the model:** Try this command and see how it reduce the model size.
```
onmt_release_model --model "model.pt" --output "model_released.pt
```
* **Use CTranslate2:** For efficient translation, consider using [CTranslate2](https://github.com/OpenNMT/CTranslate2), a fast inference engine. Check out an [example](https://gist.github.com/ymoslem/60e1d1dc44fe006f67e130b6ad703c4b).

* **Work on low-resource languages:** Find out more details about [how to train NMT models for low-resource languages](https://blog.machinetranslation.io/low-resource-nmt/).

* **Train a multilingual model:** Find out helpful notes about [training multilingual models](https://blog.machinetranslation.io/multilingual-nmt).

* **Publish a demo:** Show off your work through a [simple demo with CTranslate2 and Streamlit](https://blog.machinetranslation.io/nmt-web-interface/).
